## 04: Full Data CNN Experiments

**Course:** CSCI 6366 (Neural Networks and Deep Learning)  
**Project:** Audio Classification using CNN  
**Notebook:** Full Data CNN Experiments

## Overview

In this notebook we scale up from the small 60-sample subset used in
`03_cnn_improved.ipynb` to a much larger portion of the dataset.

**Goals:**

- Use many more of the available 610 audio files (dog/cat/bird).
- Train and compare our **baseline CNN** and a **regularized variant**
  on a proper train/validation/test split.
- Report final test accuracy, confusion matrix, and per-class metrics.
- Summarize which architecture works best when given more data.


### 1. Setup and Configuration

In this section we:
- import the same libraries used in previous notebooks,
- set global parameters (sample rate, Mel-spectrogram settings, etc.),
- configure how many files per class to use,
- and fix random seeds for reproducibility.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

import librosa
import librosa.display

import tensorflow as tf
from tensorflow.keras import layers, models

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

np.random.seed(42)
tf.random.set_seed(42)

DATA_DIR = Path("../data").resolve()

# Audio / mel-spectrogram parameters (same as previous notebooks)
SAMPLE_RATE = 16000
N_FFT = 1024
HOP_LENGTH = 512
N_MELS = 128

CLASS_NAMES = ["dog", "cat", "bird"]
label_to_index = {label: idx for idx, label in enumerate(CLASS_NAMES)}

# How many files per class to use (None = use all)
MAX_FILES_PER_CLASS = None  # or e.g. 150

# Train/val/test ratios
TEST_SIZE = 0.15
VAL_SIZE = 0.15  # of the remaining after test split


## 2. Helper Functions for Preprocessing

We reuse the same preprocessing pipeline as in previous notebooks to ensure consistency:

1. Load the raw audio file with `librosa.load` at 16 kHz.
2. Convert to a Mel-spectrogram with `librosa.feature.melspectrogram`.
3. Convert to log scale (`librosa.power_to_db`) and normalize to [0, 1].
4. Pad or crop each spectrogram to a fixed 128×128 window.
5. Add channel dimension to get shape (128, 128, 1).
6. Stack them into arrays `X` (inputs) and `y` (one-hot labels).


In [ ]:
def load_mel_spectrogram(path, sample_rate=SAMPLE_RATE, n_fft=N_FFT,
                         hop_length=HOP_LENGTH, n_mels=N_MELS):
    y, sr = librosa.load(path, sr=sample_rate)
    S = librosa.feature.melspectrogram(
        y=y,
        sr=sr,
        n_fft=n_fft,
        hop_length=hop_length,
        n_mels=n_mels,
        power=2.0,
    )
    S_db = librosa.power_to_db(S, ref=np.max)
    # Normalize roughly to [0, 1]
    S_norm = (S_db - S_db.min()) / (S_db.max() - S_db.min() + 1e-8)
    return S_norm


def pad_or_crop_spectrogram(S, target_time_bins=128):
    n_mels, n_frames = S.shape

    if n_mels != N_MELS:
        raise ValueError(f"Expected {N_MELS} Mel bands, got {n_mels}")

    if n_frames == target_time_bins:
        return S

    if n_frames < target_time_bins:
        pad_width = target_time_bins - n_frames
        S_padded = np.pad(S, ((0, 0), (0, pad_width)), mode="constant")
        return S_padded

    # If too long, centrally crop
    start = (n_frames - target_time_bins) // 2
    end = start + target_time_bins
    return S[:, start:end]


def load_example_for_model(audio_path: Path, label: str) -> tuple[np.ndarray, np.ndarray]:
    """Load one audio file and return (X, y) ready for model."""
    S_norm = load_mel_spectrogram(audio_path)
    S_fixed = pad_or_crop_spectrogram(S_norm, target_time_bins=128)

    # Add channel dimension → (128, 128, 1)
    X = S_fixed.astype("float32")[..., np.newaxis]

    # One-hot label
    num_classes = len(CLASS_NAMES)
    y = np.zeros(num_classes, dtype="float32")
    y[label_to_index[label]] = 1.0

    return X, y


def load_dataset_full(max_files_per_class: int | None = MAX_FILES_PER_CLASS):
    """Load many files per class (or all if max_files_per_class is None)."""
    X_list = []
    y_list = []

    for label in CLASS_NAMES:
        class_dir = DATA_DIR / label
        wav_files = sorted(class_dir.glob("*.wav"))

        if max_files_per_class is not None:
            wav_files = wav_files[:max_files_per_class]

        for audio_path in wav_files:
            X, y = load_example_for_model(audio_path, label)
            X_list.append(X)
            y_list.append(y)

    X = np.stack(X_list, axis=0)
    y = np.stack(y_list, axis=0)
    return X, y


## 3. Load Full Dataset

Load the dataset using the helper functions. If `MAX_FILES_PER_CLASS` is `None`, we'll use all available files. Otherwise, we'll limit to the specified number per class.


In [ ]:
X, y = load_dataset_full(MAX_FILES_PER_CLASS)
print("Dataset shapes:", X.shape, y.shape)

y_indices = np.argmax(y, axis=1)
unique, counts = np.unique(y_indices, return_counts=True)
for idx, count in zip(unique, counts):
    print(f"{CLASS_NAMES[idx]}: {count} files")


### 3.1 Dataset Summary

The dataset has been loaded with the specified number of files per class. This gives us a much larger training set compared to the initial 60-sample subset used in earlier experiments.


## 4. Train / Validation / Test Split

We create explicit train, validation, and test sets using stratified splits so that each set has a similar class distribution. This ensures fair evaluation and prevents class imbalance issues.


In [ ]:
# First split off test set
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X,
    y,
    test_size=TEST_SIZE,
    random_state=42,
    stratify=y_indices,
)

# Need class indices for stratified split again
y_train_full_idx = np.argmax(y_train_full, axis=1)

# Now split train_full into train and validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full,
    y_train_full,
    test_size=VAL_SIZE,
    random_state=42,
    stratify=y_train_full_idx,
)

print("Train shape:", X_train.shape)
print("Validation shape:", X_val.shape)
print("Test shape:", X_test.shape)

# Show class distribution in each split
print("\nClass distribution:")
for split_name, y_split in [("Train", y_train), ("Validation", y_val), ("Test", y_test)]:
    y_split_idx = np.argmax(y_split, axis=1)
    unique, counts = np.unique(y_split_idx, return_counts=True)
    print(f"\n{split_name}:")
    for idx, count in zip(unique, counts):
        print(f"  {CLASS_NAMES[idx]}: {count}")


### 4.1 Split Summary

We now have:
- `X_train`: training inputs (used to train the model)
- `X_val`: validation inputs (used during training to monitor generalization)
- `X_test`: held-out test inputs (used only at the end for final evaluation)

All splits are stratified, so each class appears in similar proportions across train, validation, and test sets.


## 5. Plotting Helper Function

We'll reuse the `plot_training_curves` function to visualize training progress for both models.


In [ ]:
def plot_training_curves(history, title_prefix=""):
    """Plot training and validation loss/accuracy."""
    history_dict = history.history

    train_loss = history_dict.get("loss", [])
    val_loss = history_dict.get("val_loss", [])
    train_acc = history_dict.get("accuracy", [])
    val_acc = history_dict.get("val_accuracy", [])

    epochs = range(1, len(train_loss) + 1)

    plt.figure(figsize=(12, 4))

    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, label="Train loss")
    if val_loss:
        plt.plot(epochs, val_loss, label="Val loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"{title_prefix} Training vs Validation Loss")
    plt.legend()

    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc, label="Train acc")
    if val_acc:
        plt.plot(epochs, val_acc, label="Val acc")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title(f"{title_prefix} Training vs Validation Accuracy")
    plt.legend()

    plt.tight_layout()
    plt.show()


## 6. Define Candidate Models

We'll train and compare two models:

1. **Baseline CNN**: The same architecture from `02_cnn_baseline.ipynb` (Dense 64, no regularization)
2. **Regularized CNN**: Baseline architecture with Dropout (0.3) added before the final classification layer

The goal is to see whether regularization helps when we have more data, compared to the small dataset experiments where Dropout(0.5) was too strong.


In [ ]:
def build_baseline_cnn(input_shape=(128, 128, 1), num_classes=3):
    """Baseline CNN architecture (same as notebook 02)."""
    model = tf.keras.Sequential([
        tf.keras.Input(shape=input_shape),

        layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),
    ])

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


def build_cnn_with_dropout(input_shape=(128, 128, 1), num_classes=3, dropout_rate=0.3):
    """Baseline CNN with Dropout regularization."""
    model = tf.keras.Sequential([
        tf.keras.Input(shape=input_shape),

        layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dropout(dropout_rate),
        layers.Dense(num_classes, activation="softmax"),
    ])

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


## 7. Train Model A: Baseline CNN

Train the baseline CNN model on the full dataset.


In [ ]:
EPOCHS = 20
BATCH_SIZE = 16

baseline_model = build_baseline_cnn()
baseline_model.summary()

baseline_history = baseline_model.fit(
    X_train,
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    verbose=1,
)

plot_training_curves(baseline_history, title_prefix="Baseline CNN")


## 8. Train Model B: CNN with Dropout

Train the regularized CNN model with Dropout(0.3) on the same dataset.


In [ ]:
drop_model = build_cnn_with_dropout()
drop_model.summary()

drop_history = drop_model.fit(
    X_train,
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    verbose=1,
)

plot_training_curves(drop_history, title_prefix="CNN + Dropout (0.3)")


## 9. Evaluate Models on Test Set

Evaluate both models on the held-out test set and generate:
- Test accuracy and loss
- Confusion matrix
- Per-class classification report (precision, recall, F1-score)


In [ ]:
def evaluate_model(model, X_test, y_test, model_name="Model"):
    """Evaluate model and print metrics."""
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"{model_name} - Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

    y_true = np.argmax(y_test, axis=1)
    y_pred = np.argmax(model.predict(X_test, verbose=0), axis=1)

    print("\nConfusion matrix:")
    cm = confusion_matrix(y_true, y_pred)
    print(cm)

    print("\nClassification report:")
    print(classification_report(y_true, y_pred, target_names=CLASS_NAMES))

    return test_loss, test_acc, cm


print("=" * 60)
print("BASELINE CNN RESULTS")
print("=" * 60)
baseline_results = evaluate_model(baseline_model, X_test, y_test, model_name="Baseline CNN")

print("\n" + "=" * 60)
print("CNN + DROPOUT (0.3) RESULTS")
print("=" * 60)
drop_results = evaluate_model(drop_model, X_test, y_test, model_name="CNN + Dropout (0.3)")


## 10. Summary of Full-Data Experiments

Here we summarize the performance of our two candidate models on the larger dataset:

- **Baseline CNN** (Dense 64, no regularization)
- **CNN with Dropout** (Dense 64 + Dropout 0.3)

### Key Findings

*(Fill in after running the experiments)*

- **Baseline CNN**:
  - Final training accuracy: ???
  - Final validation accuracy: ???
  - Test accuracy: ???
  - Test loss: ???

- **CNN + Dropout (0.3)**:
  - Final training accuracy: ???
  - Final validation accuracy: ???
  - Test accuracy: ???
  - Test loss: ???

### Discussion

**Which model generalizes better?**

*(Fill in after seeing results)*

**Did Dropout help when we have more data?**

*(Fill in after seeing results - compare to the small dataset experiments where Dropout(0.5) was too strong)*

**Which model do we choose as our final one for the project?**

*(Fill in after seeing results)*

### Comparison with Small Dataset Experiments

In `03_cnn_improved.ipynb`, we found that:
- Baseline (Dense 64) achieved ~60% val accuracy and ~42% test accuracy on 60 samples
- Dropout(0.5) was too strong and hurt performance (test acc dropped to ~33%)

With the larger dataset, we expect:
- More reliable validation metrics (less noise)
- Better overall performance due to more training data
- Dropout(0.3) may be more effective than Dropout(0.5) was on the small dataset

*(Update this section after running experiments)*
